# Univariate Anomaly Detection Demo Code (SDK Version)

In [ ]:
# pip install --upgrade azure-ai-anomalydetector

In [1]:
import os
import time
import json

from datetime import datetime
from azure.ai.anomalydetector import AnomalyDetectorClient
from azure.ai.anomalydetector.models import DetectRequest, TimeSeriesPoint, TimeGranularity, \
    AnomalyDetectorError
from azure.core.credentials import AzureKeyCredential

In [2]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

Loading BokehJS ...

In [3]:
subscription_key = '[Placeholder: Your Anomaly Detector resource access key]' 
anomaly_detector_endpoint = '[Placeholder: Your Anomaly Detector resource endpoint]'

# create an Anomaly Detector client
ad_client = AnomalyDetectorClient(AzureKeyCredential(subscription_key), anomaly_detector_endpoint)

In [4]:
def build_figure(result, sample_data, sensitivity):
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
              'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins']
              , 'value': [x['value'] for x in sample_data['series']], 'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Anomaly Detection Result ({0} Sensitivity)".format(sensitivity), width=800, height=200)
    for anom in response['isAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['expectedValues'] + response.iloc[index]['upperMargins'] or 
                         values[index] < response.iloc[index]['expectedValues'] - response.iloc[index]['lowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend_label='Boundary')
    p.line(label, values, legend_label='value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend_label='expectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

## Latest point anomaly detection with the Anomaly Detector SDK

In [5]:
def detect_anomaly_last(sample_data, ad_client, sensitivity, skip_point=29):
    points = sample_data["series"]
    granularity = sample_data["granularity"]
    result = {
        "expectedValues": [None] * len(points),
        "upperMargins": [None] * len(points),
        "lowerMargins": [None] * len(points),
        "isNegativeAnomaly": [False] * len(points),
        "isPositiveAnomaly": [False] * len(points),
        "isAnomaly": [False] * len(points)
    }
    anom_count = 0
    
    for i in range(skip_point, len(points) + 1):
        series = [TimeSeriesPoint(timestamp=item["timestamp"], value=item["value"]) for item in points[i - 29: i]]
        request = DetectRequest(series=series, granularity=granularity, sensitivity=sensitivity, max_anomaly_ratio=0.25)
        single_point = ad_client.detect_last_point(request)
        if single_point.is_anomaly == True:
            anom_count += 1
        result['expectedValues'][i-1] = single_point.expected_value
        result['upperMargins'][i-1] = single_point.upper_margin
        result['lowerMargins'][i-1] = single_point.lower_margin
        result['isNegativeAnomaly'][i-1] = single_point.is_negative_anomaly
        result['isPositiveAnomaly'][i-1] = single_point.is_positive_anomaly
        result['isAnomaly'][i-1] = single_point.is_anomaly
    return result

In [6]:
sample_data = json.load(open('./univariate_sample_daily.json'))
print(f"granularity: {sample_data['granularity']}")

granularity: daily


#### 95 sensitivity

In [7]:
sensitivity = 95
skip_point = 29  # skip the first 29 points due to insufficient data
results = detect_anomaly_last(sample_data, ad_client, sensitivity, skip_point)
build_figure(results, sample_data, sensitivity)

#### 85 sensitivity

In [8]:
sensitivity = 85
skip_point = 29  # skip the first 29 points due to insufficient data
results = detect_anomaly_last(sample_data, ad_client, sensitivity, skip_point)
build_figure(results, sample_data, sensitivity)

## Batch anomaly detection with the Anomaly Detector SDK

In [9]:
def detect_anomaly_entire(sample_data, ad_client, sensitivity):
    points = sample_data["series"]
    granularity = sample_data["granularity"]
    period = sample_data["period"] if "period" in sample_data else None
    series = [TimeSeriesPoint(timestamp=item["timestamp"], value=item["value"]) for item in points]
    request = DetectRequest(series=series, granularity=granularity, sensitivity=sensitivity, period=period)
    batch = ad_client.detect_entire_series(request)
    result = {}
    result['expectedValues'] = batch.expected_values
    result['upperMargins'] = batch.upper_margins
    result['lowerMargins'] = batch.lower_margins
    result['isNegativeAnomaly'] = batch.is_negative_anomaly
    result['isPositiveAnomaly'] = batch.is_positive_anomaly
    result['isAnomaly'] = batch.is_anomaly
    return result

### Case 1: time series with an hourly sampling frequency

In [10]:
sample_data = json.load(open('./univariate_sample_hourly.json'))
print(f"granularity: {sample_data['granularity']}")
print(f"period: {sample_data['period']}")

granularity: hourly
period: 24


#### 95 sensitivity

In [11]:
sensitivity = 95
results = detect_anomaly_entire(sample_data, ad_client, sensitivity)
build_figure(results, sample_data, sensitivity)

#### 90 sensitivity

In [12]:
sensitivity = 90
results = detect_anomaly_entire(sample_data, ad_client, sensitivity)
build_figure(results, sample_data, sensitivity)

#### 85 sensitivity

In [13]:
sensitivity = 85
results = detect_anomaly_entire(sample_data, ad_client, sensitivity)
build_figure(results, sample_data, sensitivity)

### Case 2: time series with a daily sampling frequency

In [14]:
sample_data = json.load(open('./univariate_sample_daily.json'))
print(f"granularity: {sample_data['granularity']}")

granularity: daily


#### 95 sensitivity

In [15]:
sensitivity = 95
results = detect_anomaly_entire(sample_data, ad_client, sensitivity)
build_figure(results, sample_data, sensitivity)

#### 90 sensitivity

In [16]:
sensitivity = 90
results = detect_anomaly_entire(sample_data, ad_client, sensitivity)
build_figure(results, sample_data, sensitivity)

#### 85 sensitivity

In [17]:
sensitivity = 85
results = detect_anomaly_entire(sample_data, ad_client, sensitivity)
build_figure(results, sample_data, sensitivity)